# Task 1: 
Steven is a travel blogger that intends to create a travel food series. He is looking at data from Zomato for inspiration. He wants to find restaurants that have good user ratings and interesting past events.
Steven obtained the following data:

·   	List of restaurants
https://raw.githubusercontent.com/Papagoat/brain-assessment/main/restaurant_data.json

·   	Country Code (in excel format)
https://github.com/Papagoat/brain-assessment/blob/main/Country-Code.xlsx?raw=true

__Comments__:

I used an online json viewer for easier exploratory data analysis

In [41]:
import requests
import json
import pandas as pd

#load list of restaurants
url = "https://raw.githubusercontent.com/Papagoat/brain-assessment/main/restaurant_data.json"
response = requests.get(url)
data = json.loads(response.text)
#print(data)

#load xlsx file
excel_file_path = 'Country-Code.xlsx'
country_codes_df = pd.read_excel(excel_file_path)
country_codes_df.columns = ['country_code', 'country']
#print(country_codes_df)

,country_code,country
0,1,India
1,14,Australia
2,30,Brazil
3,37,Canada
4,94,Indonesia


In [69]:
#EDA the json file
unique_restaurant_ids = set()

for results_shown in data:
    for restaurant_info in results_shown['restaurants']:
        restaurant_id = restaurant_info['restaurant']['id']
        unique_restaurant_ids.add(restaurant_id)

unique_restaurant_count = len(unique_restaurant_ids)
print(f"Number of unique restaurant IDs: {unique_restaurant_count}")

Number of unique restaurant IDs: 1300


## 1. 	Extract the following fields and store the data as restaurants.csv.

◦   	Restaurant Id

◦   	Restaurant Name

◦   	Country

◦   	City

◦   	User Rating Votes

◦   	User Aggregate Rating (in float)

◦   	Cuisines

__Steps__
1. Inspect and extract relevant information from the json
2. Putting it into a dataframe
3. Joining with the xlsx data to get the Country
4. Format and store as csv

__Assumption__

The restaurant id under Restaurants >> restaurant >> R >> id is the same as Restaurants >> restaurant >> id

In [66]:
#1.
restaurant_in_list = []
for results_shown in data:
    for restaurant_info in results_shown['restaurants']:
        restaurant_data = restaurant_info['restaurant']
        restaurant_in_list.append({
            'restaurant_id': restaurant_data['id'],
            'restaurant_name': restaurant_data['name'],
            'country_id': restaurant_data['location']['country_id'],
            'city': restaurant_data['location']['city'],
            'user_rating_votes': restaurant_data['user_rating']['votes'],
            'user_aggregate_rating': float(restaurant_data['user_rating']['aggregate_rating']),
            'cuisines': restaurant_data['cuisines']
        })
    
#2.
restaurant_in_df = pd.DataFrame(restaurant_in_list)

#3.
final_restaurant_df = restaurant_in_df.merge(country_codes_df, how='left', left_on='country_id', right_on='country_code')

#4.
final_columns = ['restaurant_id', 'restaurant_name', 'country', 'city', 'user_rating_votes', 
                 'user_aggregate_rating', 'cuisines']
final_restaurant_df = final_restaurant_df[final_columns]
final_restaurant_df.to_csv('restaurants.csv', index=False)

In [70]:
#Check number of unique restaurants == 1300

unique_restaurant_id_count = final_restaurant_df['restaurant_id'].nunique()
print(f'Number of unique restaurant IDs: {unique_restaurant_id_count}')

Number of unique restaurant IDs: 1300


2. 	Extract the list of restaurants that have past event in the month of April 2019 and store the data as restaurant_events.csv.

◦   	Event Id

◦   	Restaurant Id

◦   	Restaurant Name

◦   	Photo URL

◦   	Event Title

◦   	Event Start Date

◦   	Event End Date

Note: Populate empty values with "NA".


__Steps__:
1. Inspect and extract relevant information from the json
2. Putting it into a dataframe
3. Convert column into datetime data type
4. Filter for past event in the month of April 2019

__Assumption__:

Restaurants that have past event in the month of April 2019 mean Events with end date during the month of April 2019


In [81]:
#1.
restaurant_events_list = []
for results_shown in data:
    for restaurant_info in results_shown['restaurants']:
        restaurant_data = restaurant_info['restaurant']
        #check if there is zomato_event for 
        if 'zomato_events' in restaurant_data:
            #maybe multiple zomato_events
            for zomato_events in restaurant_data['zomato_events']:
                event_details = zomato_events['event']
                photo_url = event_details['photos'][0]['photo']['url'] if event_details['photos'] else "NA"
                restaurant_events_list.append({
                    'event_id': event_details['event_id'],
                    'restaurant_id': restaurant_data['id'],
                    'restaurant_name': restaurant_data['name'],
                    'photo_url': photo_url,
                    'event_title': event_details.get('title', "NA"),
                    'event_start_date': event_details.get('start_date', "NA"),
                    'event_end_date': event_details.get('end_date', "NA"),
                })

#2.
restaurant_events_df = pd.DataFrame(restaurant_events_list)

#3.
restaurant_events_df['event_start_date'] = pd.to_datetime(restaurant_events_df['event_start_date'], format='%Y-%m-%d')
restaurant_events_df['event_end_date'] = pd.to_datetime(restaurant_events_df['event_end_date'], format='%Y-%m-%d')

#4.
april_events_df = restaurant_events_df[(restaurant_events_df['event_end_date'] >= '2019-04-01') & (restaurant_events_df['event_end_date'] <= '2019-04-30')]
april_events_df.to_csv('restaurant_events.csv', index=False)

   event_id restaurant_id       restaurant_name  \
0    322331      18649486  The Drunken Botanist   
1    332812        308322      Hauz Khas Social   
2    336644      18856789  AIR- An Ivory Region   
3    336889      18382360                 Local   
4    336888      18382360                 Local   

                                           photo_url  \
0  https://b.zmtcdn.com/data/zomato_events/photos...   
1  https://b.zmtcdn.com/data/zomato_events/photos...   
2  https://b.zmtcdn.com/data/zomato_events/photos...   
3                                                 NA   
4                                                 NA   

                       event_title event_start_date event_end_date  
0  BackToBasic Wednesdays !!\n\n\n       2019-03-06     2019-08-28  
1      Live 20/20 Match Screenings       2019-03-29     2019-05-23  
2               Dhol Bhangra Night       2019-04-10     2019-04-11  
3               Cocktail Wednesday       2019-04-10     2019-04-10  
4          

3. 	From the dataset (restaurant_data.json), determine the threshold for the different rating text based on aggregate rating. Return aggregates for the following ratings only:

◦   	Excellent

◦   	Very Good

◦   	Good

◦   	Average

◦   	Poor


__Steps__:
1. Inspect and extract relevant information from the json
2. Putting it into a dataframe
3. Group by rating_text and aggregate Min and Max to find the threshold

__Assumption__:

Zomato uses min and max aggregates to determind the threshold for for different rating texts.

In [91]:
# Given the simplified structure for processing
restaurant_ratings_list = []
for results_shown in data:
    for restaurant_info in results_shown['restaurants']:
        restaurant_data = restaurant_info['restaurant']
        restaurant_ratings_list.append({
            'restaurant_id': restaurant_data['id'],
            'user_aggregate_rating': float(restaurant_data['user_rating']['aggregate_rating']),
            'rating_text': restaurant_data['user_rating']['rating_text']
        })



# Create a DataFrame
restaurant_ratings_df = pd.DataFrame(restaurant_ratings_list)

# Filter the DataFrame for the specified rating texts
filtered_ratings_df = restaurant_ratings_df[restaurant_ratings_df["rating_text"].isin(["Excellent", "Very Good", "Good", "Average", "Poor"])]

# Group by rating text and calculate the aggregate statistics for ratings
rating_aggregates = filtered_ratings_df.groupby("rating_text")["user_aggregate_rating"].agg(['min', 'max']).sort_values(by="min")
rating_aggregates

,min,max
rating_text,,
Poor,2.2,2.2
Average,2.5,3.4
Good,3.5,3.9
Very Good,4.0,4.4
Excellent,4.5,4.9


Threshold for Excellent rating is: Min 4.5 aggregate rating and max aggregate rating of 4.9 based on this dataset (may be up to 5.0)

Threshold for Very Good rating is: Min 4.0 aggregate rating and max aggregate rating of 4.4 based on this dataset

Threshold for Good rating is: Min 3.5 aggregate rating and max aggregate rating of 3.9 based on this dataset

Threshold for Average rating is: Min 2.5 aggregate rating and max aggregate rating of 3.4 based on this dataset

Threshold for Poor rating is: Min 2.2 aggregate rating and max aggregate rating of 2.2 based on this dataset (Range may be wider as there is only 1 restaurant with a poor rating)
